In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import keras
from numpy import random
from keras.optimizers import Adam
import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D
from keras.layers import MaxPooling2D, BatchNormalization
from keras.optimizers import SGD
# from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.applications import MobileNet, VGG16, ResNet50,DenseNet201,InceptionV3,EfficientNetB7
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report ,confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tabulate import tabulate
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* **Directory of dataset**

In [9]:
cd /content/drive/MyDrive/PBL6_Dataset

/content/drive/MyDrive/PBL6_Dataset


In [13]:
unzipPath = "./"

In [15]:
!unzip -q archive.zip -d $unzipPath

In [17]:
pwd

'/content/drive/MyDrive/PBL6_Dataset'

In [18]:
# Directory of train, validation and test

train_folder = "./Data/train"
valid_folder = "./Data/valid"
test_folder = "./Data/test"

* **Class Labels and Size**

In [19]:
print("\n\n\t\tTraining Set")
print("\t  ========================\n")
train_c = []
for folder in os.listdir(train_folder):
    train_c.append(folder)
    print("\nTrain "+ folder + " Class: ", len(os.listdir(train_folder + '/' + folder)))

print("\n\n", train_c)

print("\n\n\t\tTesting Set")
print("\t  ========================\n")
test_c = []
for folder in os.listdir(test_folder):
    test_c.append(folder)
    print("\nTest "+ folder + " Class: ", len(os.listdir(test_folder + '/' + folder)))

print("\n\n", test_c)

print("\n\n\t\tValidation Set")
print("\t  ========================\n")
val_c = []
for folder in os.listdir(valid_folder):
    val_c.append(folder)
    print("\nValidation "+ folder + " Class: ", len(os.listdir(valid_folder + '/' + folder)))

print("\n\n", val_c, "\n")



		Training Set


Train adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib Class:  195

Train large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa Class:  115

Train normal Class:  148

Train squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa Class:  155


 ['adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa', 'normal', 'squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa']


		Testing Set


Test adenocarcinoma Class:  120

Test large.cell.carcinoma Class:  51

Test normal Class:  54

Test squamous.cell.carcinoma Class:  90


 ['adenocarcinoma', 'large.cell.carcinoma', 'normal', 'squamous.cell.carcinoma']


		Validation Set


Validation adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib Class:  23

Validation large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa Class:  21

Validation normal Class:  13

Validation squamous.cell.carcinoma_left.hilum_T1_N2_M0_IIIa Class:  15


 ['adenocarcinoma_left.lower.lobe_T2_N0_M0_Ib', 'large.cell.carcinoma_left.hilum_T2_N2_M0_IIIa', 'normal'

* **Define function to get Data**

In [20]:
input_shape = (224,224,3)
num_classes = 4

train_datagen = ImageDataGenerator(
    dtype='float32',
    preprocessing_function=preprocess_input,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
)
val_datagen = ImageDataGenerator(
    dtype='float32',
    preprocessing_function=preprocess_input,
)
test_datagen = ImageDataGenerator(
    dtype='float32',
    preprocessing_function=preprocess_input,
)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
)


test_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    shuffle = False,
)
validation_generator = val_datagen.flow_from_directory(
    valid_folder,
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
)

Found 613 images belonging to 4 classes.
Found 315 images belonging to 4 classes.
Found 72 images belonging to 4 classes.


ResNet50

In [21]:
ResNet50_model = ResNet50(weights = 'imagenet',
                     include_top = False,
                     input_shape = input_shape)

for layer in ResNet50_model.layers:
    layer.trainable = False

94765736/94765736 [==============================] - 3s 0us/step


In [22]:
# Define a Checkpoint
checkpoint = ModelCheckpoint(filepath='C_CT_S.h5',
                             monitor='val_accuracy',
                             mode='max',
                             save_best_only=True,
                             verbose=1)

# Define Early Stopping
earlystop = EarlyStopping(monitor='val_accuracy',
                         min_delta=0.001,  # Adjust the min_delta value as needed
                         patience=15,
                         restore_best_weights=True)

# Define LR Reducing Rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss',  # You can use 'val_loss' for learning rate reduction
                              factor=0.1,
                              patience=10,
                              verbose=1,
                              min_delta=0.0001,
                              min_lr=0.0001)

# Put the callbacks in a callback list
callbacks = [reduce_lr]

# Define and compile your model
model = Sequential([
    ResNet50_model,
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model with callbacks
epochs = 50

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    callbacks=callbacks,  # Include the defined callbacks
    verbose=1
)
test = test_generator
score = model.evaluate(test, verbose = 1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Epoch 1/50
20/20 [==============================] - 33s 742ms/step - loss: 4.5346 - accuracy: 0.3964 - val_loss: 2.3743 - val_accuracy: 0.4861 - lr: 0.0010
Epoch 2/50
20/20 [==============================] - 14s 700ms/step - loss: 3.1491 - accuracy: 0.4992 - val_loss: 1.3611 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 3/50
20/20 [==============================] - 15s 723ms/step - loss: 1.8659 - accuracy: 0.5008 - val_loss: 1.1886 - val_accuracy: 0.4306 - lr: 0.0010
Epoch 4/50
20/20 [==============================] - 13s 690ms/step - loss: 1.4657 - accuracy: 0.5416 - val_loss: 1.6184 - val_accuracy: 0.3750 - lr: 0.0010
Epoch 5/50
20/20 [==============================] - 14s 675ms/step - loss: 1.1764 - accuracy: 0.5498 - val_loss: 1.0245 - val_accuracy: 0.4444 - lr: 0.0010
Epoch 6/50
20/20 [==============================] - 14s 695ms/step - loss: 1.0587 - accuracy: 0.5661 - val_loss: 0.8700 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 7/50
20/20 [==============================] - 16s 813ms/st

In [23]:
model.save('Tranfer_best_model_075.h1')

In [24]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=epochs,
    callbacks=callbacks,  # Include the defined callbacks
    verbose=1
)

Epoch 1/50
20/20 [==============================] - 14s 691ms/step - loss: 0.2192 - accuracy: 0.9184 - val_loss: 0.3687 - val_accuracy: 0.8889 - lr: 0.0010
Epoch 2/50
20/20 [==============================] - 15s 762ms/step - loss: 0.2721 - accuracy: 0.8989 - val_loss: 0.4946 - val_accuracy: 0.8611 - lr: 0.0010
Epoch 3/50
20/20 [==============================] - 14s 653ms/step - loss: 0.2018 - accuracy: 0.9201 - val_loss: 0.4822 - val_accuracy: 0.8611 - lr: 0.0010
Epoch 4/50
20/20 [==============================] - 14s 664ms/step - loss: 0.2561 - accuracy: 0.9119 - val_loss: 0.5677 - val_accuracy: 0.8611 - lr: 0.0010
Epoch 5/50
20/20 [==============================] - 14s 692ms/step - loss: 0.1944 - accuracy: 0.9266 - val_loss: 0.5235 - val_accuracy: 0.8750 - lr: 0.0010
Epoch 6/50
20/20 [==============================] - 16s 800ms/step - loss: 0.2614 - accuracy: 0.9103 - val_loss: 0.3808 - val_accuracy: 0.8611 - lr: 0.0010
Epoch 7/50
20/20 [==============================] - 14s 711ms/st

In [25]:
score = model.evaluate(test, verbose = 1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

10/10 [==============================] - 3s 279ms/step - loss: 0.4474 - accuracy: 0.8571
Test loss: 0.4473704993724823
Test accuracy: 0.8571428656578064


In [26]:
model.save('Tranfer_best_model_085.h2')